In [12]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import os
import datetime

In [13]:
# Set up the file path for the input Excel file
file_path = '/Users/paraggupta/Desktop/projects/TVD_Associates/Data/Dataset_Full.xlsx'

# Read the Excel file into a DataFrame
data = pd.read_excel(file_path)

In [14]:
# Identify and convert numeric columns to numeric data type
numeric_cols = ['Amount Paid', 'Balance']
data[numeric_cols] = data[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
# Convert the 'Date Registered' column to datetime
data['Date Registered'] = pd.to_datetime(data['Date Registered'], errors='coerce')
#Calculate the nimber of day since
data['Days_Registered'] = (datetime.datetime.today() - data['Date Registered']).dt.days

In [26]:
#grouping together zipcodes
zip_series = pd.Series(data['Address Main | Zip'])
zip_count = zip_series.value_counts()

#A list of all the indices and the a list of all their counts
zip_code_names = zip_count.index
zip_code_counts = zip_count.values